In [ ]:
!pip install rouge_score
!pip install evaluate
import numpy as np
import pandas as pd
import kagglehub
import os
import kagglehub
import re
from datasets import Dataset
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))




path = kagglehub.dataset_download("gowrishankarp/newspaper-text-summarization-cnn-dailymail")

print("Path to dataset files:", path)

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8b05bcd98b96f7d5b4fc21f865951b22ca9496213633d15fa09592ad6cca6d1c
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.0 MB/s eta 0:00:00
Using Colab cache for faster access to the 'newspaper-text-summarization-cnn-dailymail' dataset.
Path to dataset files: /kaggle/input/newspaper-text-summarization-cnn-dailymail


In [ ]:
training_set = pd.read_csv('/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv')
test_set = pd.read_csv('/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv')
validation_set = pd.read_csv('/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv')
training_set.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [ ]:
training_set.drop(columns = ['id'],inplace = True)
test_set.drop(columns = ['id'],inplace = True)
validation_set.drop(columns = ['id'],inplace = True)

In [ ]:
print(training_set.isnull().sum())
print(training_set.shape)

article       0
highlights    0
dtype: int64
(287113, 2)


In [ ]:
training_set.drop_duplicates(inplace = True)

In [ ]:
print(training_set.shape)
print(validation_set.shape)
print(test_set.shape)

(284015, 2)
(13368, 2)
(11490, 2)


In [ ]:
training_set = training_set.sample(n=40_000, random_state=42)

In [ ]:
training_set.shape

(40000, 2)

In [ ]:
def clean_text(text):
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
training_set['article'] = training_set['article'].apply(clean_text)
training_set['highlights'] = training_set['highlights'].apply(clean_text)

validation_set['article'] = validation_set['article'].apply(clean_text)
validation_set['highlights'] = validation_set['highlights'].apply(clean_text)

test_set['article'] = test_set['article'].apply(clean_text)
test_set['highlights'] = test_set['highlights'].apply(clean_text)

In [ ]:
train_dataset = Dataset.from_pandas(training_set)
test_dataset = Dataset.from_pandas(test_set)
validation_dataset = Dataset.from_pandas(validation_set)

In [ ]:
model_name = 't5-base'
tokenizer = AutoTokenizer.from_pretrained(model_name,use_fast = True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
max_input_length = 512
max_target_length = 128

def preprocess_data(data):
    model_inputs = tokenizer(
        data["article"],
        max_length=max_input_length,
        truncation=True,
        padding="max_length"
    )

    labels = tokenizer(
        data["highlights"],
        max_length=max_target_length,
        truncation=True,
        padding="max_length"
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
train_dataset = train_dataset.map(preprocess_data, batched=True)
validation_dataset   = validation_dataset.map(preprocess_data, batched=True)
test_dataset  = test_dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
transformers.logging.set_verbosity_error()
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    result = {k: v.mid.fmeasure if hasattr(v, "mid") else v for k, v in result.items()}
    return result

In [ ]:
model.gradient_checkpointing_enable()

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3,
    predict_with_generate=True,
    fp16=False,
    logging_dir="./logs",
    logging_steps=50,
    warmup_ratio=0.1,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,
    disable_tqdm=False
)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

model.save_pretrained("/kaggle/working/model")
tokenizer.save_pretrained("/kaggle/working/model")

/tmp/ipython-input-3618173299.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss,Validation Loss


In [ ]:
training_args.num_train_epochs += 1
training_args.learning_rate = 1e-5
training_args.gradient_accumulation_steps = 8
training_args.eval_steps = 625
training_args.save_steps = 625


trainer.train(resume_from_checkpoint=True)

model.save_pretrained("/kaggle/working/model_epoch4")
tokenizer.save_pretrained("/kaggle/working/model_epoch4")

In [ ]:
print("Evaluation started...")
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Evaluation finished.")
print(f"Test Metrics = {metrics}")

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = T5Tokenizer.from_pretrained("/kaggle/working/model_epoch4")
model = T5ForConditionalGeneration.from_pretrained("/kaggle/working/model_epoch4")

model.to(device)

trainer.model = model

print("Evaluation started...")
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Evaluation finished.")
print(f"Test Metrics = {metrics}")